## Design of Multidimensional Kalman Filters

From this point forward we will transition to using linear algebra, as it allows us to easily work with multi-dimensional problems. To begin with, let’s write the state prediction in linear algebra form.

## State Transition

The formula below is the state transition function that advances the state from time _t_ to time _t + 1_. It is just the relationship between the robot’s position, $x$, and velocity, $\dot{x}$. Here, we will assume that the robot’s velocity is not changing.

$$
\large x' = x + \Delta t \dot{x} \\
\large \dot{x}' = \dot{x}
$$

We can express the same relationship in matrix form, as seen below. On the left, is the posterior state (denoted with the prime symbol, ' ), and on the right are the state transition function and the prior state. This equation shows how the state changes over the time period, $\Delta t$. Note that we are only working with the means here; the covariance matrix will appear later.

$$
\large \begin{bmatrix} x \\ \dot{x} \end{bmatrix}' = \begin{bmatrix} 1 & \Delta{t} \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x \\ \dot{x} \end{bmatrix}
$$

The State Transition Function is denoted FF, and the formula can be written as so,

$$
\large x' = Fx
$$

In reality, the equation should also account for process noise, as its own term in the equation. However, process noise is a Gaussian with a mean of 0, so the update equation for the mean need not include it.

$$
\large x' = Fx + noise \\
\large noise \sim N(0,Q)
$$

Now, what happens to the covariance? How does it change in this process?

Sidenote: While it is common to use $\Sigma$ to represent the covariance of a Gaussian distribution in mathematics, it is more common to use the letter $P$ to represent the state covariance in localization.

If you multiply the state, $x$, by $F$, then the covariance will be affected by the square of $F$. In matrix form, this will look like so:

$$
\large P' = FPF^T
$$
 
However, your intuition may suggest that it should be affected by more than just the state transition function. For instance, additional uncertainty may arise from the prediction itself. If so, you’re correct!

To calculate the posterior covariance, the prior covariance is multiplied by the state transition function squared, and QQ added as an increase of uncertainty due to process noise. QQ can account for a robot slowing down unexpectedly, or being drawn off course by an external influence.

$$
\large P' = FPF^T + Q
$$

Now we’ve updated the mean and the covariance as part of the state prediction.

## Quiz 1

Now that you've seen how a simple state transition function is created, let's see if you can construct a more complicated one for the following problem:

You are tracking the position and velocity of a robot in two dimensions, x and y. The state is represented as so,

$$
\large x = \begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}
$$

Find the state update function, F, that will advance the state from time _t_ to time _t + 1_ based on the state transition equation below.

$$
\large x' = Fx
$$

Try to work through this on paper before looking at the quiz options below.

> Which of the following matrices is the correct state transition function for the problem defined above?
>
> ![](images/quiz-1.png)
>
> That's right! You've got the hang of this. The velocity is left unchanged, and the new position is equal to the previous position plus the velocity multiplied by the elapsed time.

## Quiz 2

You are tracking the position, velocity, and acceleration of a quadrotor in the vertical dimension, z. The state of the quadrotor can be represented as so,

$$
\large x = \begin{bmatrix} z \\ \dot{z} \\ \ddot{z} \end{bmatrix}
$$

Find the state update function, $F$, that will advance the state from time _t_ to time _t + 1_ based on the state transition equation below.

$$
\large x' = Fx
$$

> Which of the following matrices is the correct state transition function for the problem defined above?
>
> ![](images/quiz-2.png)
>
> Great work!

## Measurement Update

Next, we move onto the measurement update step. If we return to our original example, where we were tracking the position and velocity of a robot in the x-dimension, the robot was taking measurements of the location only (the velocity is a hidden state variable). Therefore the measurement function is very simple - a matrix containing a one and a zero. This matrix demonstrates how to map the state to the observation, $z$.

$$
\large z = \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} x \\ \dot{x} \end{bmatrix}
$$

This matrix, called the Measurement Function, is denoted $H$.

For the measurement update step, there are a few formulas. First, we calculate the measurement residual, $y$. The measurement residual is the difference between the measurement and the expected measurement based on the prediction (ie. we are comparing where the measurement tells us we are vs. where we think we are). The measurement residual will be used later on in a formula.

$$
\large y = z - Hx'
$$

Next, it's time to consider the measurement noise, denoted $R$. This formula maps the state prediction covariance into the measurement space and adds the measurement noise. The result, $S$, will be used in a subsequent equation to calculate the Kalman Gain.

$$
\large S = HP'H^T + R
$$

These equations need not be memorized, instead they can be referred to in text or implemented in code for use and reuse.

## Kalman Gain

Next, we calculate the Kalman Gain, K. As you will see in the next equation, the Kalman Gain determines how much weight should be placed on the state prediction, and how much on the measurement update. It is an averaging factor that changes depending on the uncertainty of the state prediction and measurement update.

$$
\large K = P'H^TS^{-1} \\
\large x = x' + Ky
$$

These equations may look complicated and intimidating, but they do nothing more than calculate an average factor.

The last step in the Kalman Filter is to update the new state’s covariance using the Kalman Gain.

$$
\large P = (I - KH)P'
$$

# Kalman Filter Equations

These are the equations that implement the Kalman Filter in multiple dimensions.

State Prediction:

$$
\large x' = Fx \\
\large P' = FPF^T + Q
$$

Measurement Update:

$$
\large y = z - Hx' \\
\large S = HP'H^T + R
$$

Calculation of Kalman Gain:

$$
\large K = P'H^TS^{-1}
$$
 
Calculation of Posterior State and Covariance:

$$
\large x = x' + Ky \\
\large P = (I - KH)P'
$$ 

The Kalman Filter can successfully recover from inaccurate initial estimates, but it is very important to estimate the noise parameters, Q and R, as accurately as possible - as they are used to determine which of the estimate or the measurement to believe more.

## Programming Exercise

Now it’s your chance to code the multi-dimensional Kalman Filter in C++. The code below uses the C++ `eigen` library to define matrices and easily compute their inverse and transpose. Check out the `eigen` library full documentation [here](https://eigen.tuxfamily.org/dox/group__QuickRefPage.html) and go through some of their examples. Here's a list of useful commands that you'll need while working on this quiz:

- Initializing a 2x1 float matrix **K**: `MatrixXf K(2, 1)`;
- Inserting values to matrix **K**: `K << 0, 0`
- Computing the transpose of matrix **K**: `K.transpose()`
- Computing the inverse of matrix **K**: `K.inverse()`

In [1]:
#pragma cling add_include_path("/usr/include/eigen3/")

In [2]:
#include <iostream>
#include <math.h>
#include <tuple>
#include <Eigen/Core>
#include <Eigen/LU>

In [3]:
float measurements[3] = { 1, 2, 3 };

In [4]:
std::tuple<Eigen::MatrixXf, Eigen::MatrixXf> kalman_filter(
    Eigen::MatrixXf x, Eigen::MatrixXf P, Eigen::MatrixXf u, 
    Eigen::MatrixXf F, Eigen::MatrixXf H, Eigen::MatrixXf R, 
    Eigen::MatrixXf I)
{
    for (int n = 0; n < sizeof(measurements) / sizeof(measurements[0]); n++) {

        // Measurement Update
        Eigen::MatrixXf Z(1, 1);
        Z << measurements[n];

        Eigen::MatrixXf y(1, 1);
        y << Z - (H * x);

        Eigen::MatrixXf S(1, 1);
        S << H * P * H.transpose() + R;

        Eigen::MatrixXf K(2, 1);
        K << P * H.transpose() * S.inverse();

        x << x + (K * y);

        P << (I - (K * H)) * P;

        // Prediction
        x << (F * x) + u;
        P << F * P * F.transpose();
    }

    return std::make_tuple(x, P);
}

In [5]:
void run()
{
    Eigen::MatrixXf x(2, 1);// Initial state (location and velocity) 
    x << 0,
    	 0; 
    Eigen::MatrixXf P(2, 2);//Initial Uncertainty
    P << 100, 0, 
    	 0, 100; 
    Eigen::MatrixXf u(2, 1);// External Motion
    u << 0,
    	 0; 
    Eigen::MatrixXf F(2, 2);//Next State Function
    F << 1, 1,
    	 0, 1; 
    Eigen::MatrixXf H(1, 2);//Measurement Function
    H << 1,
    	 0; 
    Eigen::MatrixXf R(1, 1); //Measurement Uncertainty
    R << 1;
    Eigen::MatrixXf I(2, 2);// Identity Matrix
    I << 1, 0,
    	 0, 1; 

    std::tie(x, P) = kalman_filter(x, P, u, F, H, R, I);
    std::cout << "x= " << x << std::endl;
    std::cout << "P= " << P << std::endl;

    return 0;
}

In [6]:
run()

x=  3.99664
0.999984
P=  2.31904  0.99176
 0.99176 0.495058
